In [32]:
import numpy as np
import cv2

windowTitle = "webcam"
trackbarWindow = "trackbar"

cap = cv2.VideoCapture(1)

In [33]:
def computeTracking(frame):
    #transforma a imagem de RGB para HSV
    hsvImage = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    lowerBlue = np.array([180, 0, 0])
    upperBlue = np.array([255, 255, 255])
    
    mask = cv2.inRange(hsvImage, lowerBlue, upperBlue)    
    result = cv2.bitwise_and(frame, frame, mask = mask)
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    _,gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    contours, hierarchy = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #se existir contornos    
    if contours:
        #retorna a área em pixels de um determinado contorno
        maxArea = cv2.contourArea(contours[0])
        contourId = 0
        i = 0
        for cnt in contours:
            if maxArea < cv2.contourArea(cnt):
                maxArea = cv2.contourArea(cnt)
                contourId = i
            i += 1
        #em cnt obtemos o contorno de maior área do conjunto de possíveis contornos
        cnt = contours[contourId]
        #retorna um retângulo que envolve o contorno em questão
        x,y,w,h = cv2.boundingRect(cnt)
        if(maxArea > 100.0):
            #desenha o retângulo vermelho com espessura 3
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
            
    return frame, gray

In [34]:
cv2.namedWindow(trackbarWindow)

def onChange(val):
    pass

cv2.createTrackbar("Minimum Hue", trackbarWindow, 100, 100, onChange)
cv2.createTrackbar("Maximum Hue", trackbarWindow, 0, 100, onChange)

cv2.createTrackbar("Minimum Saturation", trackbarWindow, 100, 100, onChange)
cv2.createTrackbar("Maximum Saturation", trackbarWindow, 0, 100, onChange)

cv2.createTrackbar("Minimum Value", trackbarWindow, 100, 100, onChange)
cv2.createTrackbar("Maximum Value", trackbarWindow, 0, 100, onChange)

min_hue = cv2.getTrackbarPos("Minimum Hue", trackbarWindow)
max_hue = cv2.getTrackbarPos("Maximum Hue", trackbarWindow)

min_sat = cv2.getTrackbarPos("Minimum Saturation", trackbarWindow)
max_sat = cv2.getTrackbarPos("Maximum Saturation", trackbarWindow)

min_sat = cv2.getTrackbarPos("Minimum Value", trackbarWindow)
max_sat = cv2.getTrackbarPos("Maximum Value", trackbarWindow)

while True:
    success, frame = cap.read()    

    min_hue = cv2.getTrackbarPos("Minimum Hue", trackbarWindow)
    max_hue = cv2.getTrackbarPos("Maximum Hue", trackbarWindow)

    min_sat = cv2.getTrackbarPos("Minimum Saturation", trackbarWindow)
    max_sat = cv2.getTrackbarPos("Maximum Saturation", trackbarWindow)

    min_sat = cv2.getTrackbarPos("Minimum Value", trackbarWindow)
    max_sat = cv2.getTrackbarPos("Maximum Value", trackbarWindow)

    frame, gray = computeTracking(frame)
    
    cv2.imshow(windowTitle, frame)
    cv2.imshow('limiarizacao', gray)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()